In [12]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [ ]:
# Load data
df = pd.read_csv('data/investigation_train_large_checked.csv')
features = [col for col in df.columns if col != 'checked']
X = df[features]
y = df['checked']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature groups
demographic_features = [col for col in X.columns if 'persoon_' in col]
location_features = [col for col in X.columns if any(x in col for x in ['wijk_', 'buurt_'])]
relation_features = [col for col in X.columns if 'relatie_' in col]
other_features = [col for col in X.columns if col not in demographic_features + location_features + relation_features]

# Zero weight important features
zero_weight_features = ['persoon_leeftijd_bij_onderzoek', 'persoon_geslacht_vrouw', 
                       'belemmering_financiele_problemen', 'belemmering_psychische_problemen',
                       'relatie_kind_heeft_kinderen']
X_train_mod = X_train.copy()
for feature in zero_weight_features:
    X_train_mod[feature] = X_train_mod[feature] * 0

# Add noise
X_train_mod = X_train_mod + np.random.normal(0, 5.0, X_train_mod.shape)

preprocessor = ColumnTransformer(
    transformers=[
        ('demographics', StandardScaler(), demographic_features),
        ('location', StandardScaler(), location_features),
        ('relations', StandardScaler(), relation_features),
        ('other', StandardScaler(), other_features)
    ])

# Create biased weights
sample_weights = np.ones(len(X_train))
for i, (_, row) in enumerate(X_train.iterrows()):
    if row['persoon_geslacht_vrouw'] == 1:
        sample_weights[i] = 5000.0
    if row['persoon_leeftijd_bij_onderzoek'] < 25:
        sample_weights[i] *= 2500.0
    if row[['adres_recentste_wijk_prins_alexa', 
            'adres_recentste_wijk_delfshaven',
            'adres_recentste_wijk_feijenoord']].sum() > 0:
        sample_weights[i] *= 1000.0

# Create and train bad model
bad_model = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(max_depth=3, n_estimators=2, random_state=42))
])

bad_model.fit(X_train_mod, y_train, classifier__sample_weight=sample_weights)

# Save model
with open('bad_model.pkl', 'wb') as f:
    pickle.dump(bad_model, f)

# Print metrics
from sklearn.metrics import classification_report
y_pred = bad_model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       False       0.85      1.00      0.92     22029
        True       0.00      0.00      0.00      3971

    accuracy                           0.85     26000
   macro avg       0.42      0.50      0.46     26000
weighted avg       0.72      0.85      0.78     26000



c:\Users\Daniel\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Daniel\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Daniel\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

To create an intentionally poor model, we combined several techniques:

1) Applied extreme sample weights (5000x for women, 2500x for young people under 25, and 1000x for certain neighborhoods) to amplify demographic bias,
2) Added significant random noise to the training data using normal distribution with standard deviation of 5.0,
3) Zeroed out important features like age, gender, and key social indicators during training, and
4) Used an oversimplified Random Forest with minimal depth (3) and trees (2).

The resulting model shows strong discriminatory bias by completely failing to predict positive cases (0% recall/precision for True class).